In [10]:
import os, re, sys
from scripts.tools.issuetracker import IssuetrackerAPI
from datetime import date
from officelib.xllib import *
import importlib
try:
    importlib.reload(jpnotebooks.Software.SDLC_traceability_tools.issuetracker_item_extracter)
except NameError:
    pass
import jpnotebooks.Software.SDLC_traceability_tools.issuetracker_item_extracter as iie
from jpnotebooks.Software.SDLC_traceability_tools.user_tests_parser import UserTestsParser
RequirementExtracter = iie.RequirementExtracter

In [3]:
_cache = None
_api = None
_sync_date = None
def _download(**kw):
    global _cache, _api, _sync_date
    if _api is None:
        _api = IssuetrackerAPI('issue.pbsbiotech.com', 'nstarkweather', 'kookychemist')
    _cache = _api.download_issues(project_id="pbssoftware", **kw)
    _sync_date = date.today()
    
def download(force=False, **kw):
    global _cache, _sync_date
    reason = None
    if _cache is None:
        reason = "cache is None"
    elif force:
        reason = "forced re-cache"
    elif _sync_date is None:
        reason = "can't determine last sync time"
    elif date.today() != _sync_date:
        reason = "cache age > 0 days"
    if reason:
        print("Downloading issues:", reason)
        _download(**kw)
    return _cache.copy()

def load_issues(**kw):
    return download(**kw)

In [17]:
rtags = [
    "URS",
    "FRS",
    "SDS",
    "CS",
    "BUG"
]

ttags = [
    "USR"
]

_ignore = {3194, 3287}
def relevant(i):
    return (i.sprint_milestone == "Legacy" or i.sprint_milestone == '3.0') and i.id not in _ignore and i.status != "Rejected"


issues = load_issues()
issues = list(filter(relevant, issues.values()))
parser = iie.IssuetrackerParser(tags)
reqs = parser.parse_all(issues)
parser2 = UserTestsParser()
tests = parser2.parse_excel("C:\\Users\\Nathan\\Documents\\Dropbox\\FRS\\FRS Tests 190225.xlsx")
rows = RequirementExtracter(rtags, ttags).extract(reqs + tests)

ValueError: Can't find item 'FRS3664.4.3' referenced by 'USR2521'

In [13]:
def paste(cr, c1, data):
    c2 = c1.Offset(len(data), len(data[0]))
    cr(c1, c2).Value = data

In [14]:
xl, wb = xlBook2()
ws = wb.Worksheets(1)
cells = ws.Cells
cr = cells.Range
paste(cr, cells(1,1), rows)